In [ ]:
# remover weekday da tabela, deixar la so de leitura

In [ ]:
from ruptura.RupturaTable import RupturaTable
from ruptura.CreateBatch import CreateBatch
import collections

In [ ]:
rupturaTable = RupturaTable()
files = ['barbieri-only.csv']  #concat all files here
clipData = ['01/11/2018', '01/04/2019']
rupturaTable.generate(files,clipData)

In [ ]:
%%time
createBatch = CreateBatch('0-1-0')
amostras = createBatch.create(rupturaTable.getData(), 'BARBIERI', '01/03/2019')
#createBatch.exportBatch(amostras, 'barbieri-ymod.json')

In [ ]:
amostras['Mart Minas Paracatu Loja 224-BISCOITO BARBIERI ESCALDADO 150G']

# PAREI AQUI

In [ ]:
import datetime
from ruptura.TableEnrichment import TableEnrichment
# dado uma data qualquer ---> pra isso tem que transformar esse cara (-160 e um numero conheciudo)
# pegar as ultimas N visitas para um determinado produto (n=5,10 etc.)
# contruir um X assim [x1,data1, x2,data2, x3,data3 ... ]
def dateStringToDatetime(dateI):
    momento = dateI.split('/')
    dia = int(momento[0])
    mes = int(momento[1])
    ano = int(momento[2].split(' ')[0])
    return datetime.datetime(ano, mes, dia)

In [ ]:
referenceVisitDay = '02/03/2019'
referenceVisitDay = dateStringToDatetime(referenceVisitDay)

In [82]:
tableEnrich = TableEnrichment('0-2-1')
visits = tableEnrich.generate(amostras)
visits['Mart Minas Paracatu Loja 224']

In [ ]:
from ruptura.EncoderX import EncoderX
encoderX = EncoderX('0-1-0')
encoderX.oneHot(visitas['Mart Minas Paracatu Loja 224'][-160]['BISCOITO BARBIERI ESCALDADO 150G'])

# GERANDO TABELA PARA CONFERENCIA

In [ ]:
# APENAS BARBIERI
indBarbieri = []
for i in data.index:
    if data.Produto[i] in todosBiscoitos:
        indBarbieri.append(i)
dataBarbieri = data.iloc[indBarbieri,:].copy()
dataBarbieri = dataBarbieri.reset_index(drop=True)
dataBarbieri.to_csv('data-barbieri.csv',sep=';')